In [152]:
import configparser
from datetime import datetime
import os
import pandas as pd
from pyspark.sql.types import TimestampType, DateType
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession 
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, dayofweek, date_format

## Import AWS Key and Secret into OS variables

In [153]:
config = configparser.ConfigParser()
config.read('dl.cfg')
os.environ['AWS_ACCESS_KEY_ID']=config['IAM']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['IAM']['AWS_SECRET_ACCESS_KEY']
key = os.getenv("AWS_ACCESS_KEY_ID")
secret = os.getenv("AWS_SECRET_ACCESS_KEY")

# Create Spark Session

In [154]:
spark = SparkSession \
        .builder \
        .appName("Sparkify_Lake") \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .config("spark.hadoop.fs.s3a.fast.upload","true")\
        .getOrCreate()

In [ ]:
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.access.key", key)
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.secret.key", secret)

# Load Songs and Logs Data

In [ ]:
load_songs = "s3a://udacity-dend/song_data/A/A/A/TRAAAAK128F9318786.json"
load_logs =  "s3a://udacity-dend/log_data/2018/11/2018-11-01-events.json"

In [ ]:
## Spark reads the song data and prints it's schema
song_data = spark.read.json(load_songs)
song_data.printSchema()

In [ ]:
## Spark reads the log data and prints it's schema
log_data = spark.read.json(load_logs)
log_data.printSchema()